In [1]:
%load_ext autoreload
%autoreload 2
from benchmark.utils import *
from benchmark.preprocess import *
from benchmark.constants import *
from benchmark.db_import import *
from benchmark.ground_truth import *
import logging
#from benchmark import sql_api
import configparser
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    filename='app.log',
    #filemode='w'
    filemode='a',
)
logging.info('This will get logged to a file')

In [2]:

config_dir = "/Users/robinbuchta/Documents/GitHub/trustathsh/projekte/secder/Fachliches/darpa_benchmark_lite_sql/configs"
config_file_record_01 = config_dir + '/eng3/cadets/cadets_01_record.ini'
config = configparser.ConfigParser()
config.read(config_file_record_01)

['/Users/robinbuchta/Documents/GitHub/trustathsh/projekte/secder/Fachliches/darpa_benchmark_lite_sql/configs/eng3/cadets/cadets_01_record.ini']

In [3]:
import_nodes(config)

In [4]:
import_edges(config)

In [3]:
input_file = config['Files']['tar']
output_dir = config['Directories']['output']
unzipped_dir = config['Directories']['unzipped']

In [23]:
print(output_dir)

/Users/robinbuchta/Documents/DARPA/cadets/01_record/output/


In [4]:
#create folder, if not exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    logging.info(f'Created output folder {output_dir}')

In [5]:
# unzip tar.gz file
unzip_tar_gz(input_file, unzipped_dir)

In [6]:
rename_files(unzipped_dir)

Renamed 'ta1-cadets-e3-official.json.2' to 'ta1-cadets-e3-official.json.02'
Renamed 'ta1-cadets-e3-official.json.1' to 'ta1-cadets-e3-official.json.01'


In [10]:
file_list = find_json_files(unzipped_dir)

In [8]:
type(file_list)

list

In [12]:
node_keys, edge_keys = get_and_write_keys(file_list)

In [13]:
print(type(node_keys))
print(type(edge_keys))

<class 'set'>
<class 'set'>


In [19]:
#get first value of node_keys
len(node_keys)

94

In [22]:
process_json_files(file_list, output_dir, node_keys, edge_keys)

In [25]:
nodes_file = output_dir + 'nodes.csv'
edges_file = output_dir + 'edges.csv'

In [26]:
nodes_uuid_column_index, node_mapping = get_node_mapping(output_dir, nodes_file)


In [27]:
print(type(nodes_uuid_column_index))
print(type(node_mapping))

<class 'dict'>
<class 'dict'>


In [28]:
edges_uuid_column_index =  find_uuid_column_index(edges_file)

In [32]:
for key in nodes_uuid_column_index:
    print(key)
    print(nodes_uuid_column_index[key])

datum.com.bbn.tc.schema.avro.cdm18.Host.uuid
18
datum.com.bbn.tc.schema.avro.cdm18.Subject.uuid
31
datum.com.bbn.tc.schema.avro.cdm18.SrcSinkObject.uuid
46
datum.com.bbn.tc.schema.avro.cdm18.NetFlowObject.uuid
63
datum.com.bbn.tc.schema.avro.cdm18.FileObject.uuid
76
datum.com.bbn.tc.schema.avro.cdm18.Principal.uuid
80
datum.com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject.uuid
85


In [33]:
conn = establish_connection(config)

In [34]:
node_file = config['Directories']['output'] + 'updated_nodes.csv'


In [35]:
columns_with_type = prepare_columns(node_file, n=10**5) 


In [37]:
type(columns_with_type)

str

In [ ]:
create_table(conn, config, columns_with_type)


In [38]:
load_data_infile(conn, config, edges=True)

In [ ]:
import_nodes(config)

In [5]:
node_set = set()
file_path = config['Directories']['output'] + "atk_nodes.txt"

In [7]:
file_path

'/Users/robinbuchta/Documents/DARPA/cadets/01_record/output/atk_nodes.txt'

In [8]:
if os.path.exists(file_path):
    print("File exists.")
    with open(file_path, 'r') as file:
        for line in file:
            node_set.add(line.strip())
else:
    logging.info(f"Getting atk nodes from config: {config}")
    yaml_data = read_gt_yaml(config)
    logging.info(f"yaml_data: {yaml_data}")
    node_set = get_atk_nodes_from_yaml(config, yaml_data, with_edges = True, only_subject=False) ## here TRUE OR FALSE FOR GT 
    #logging.info(f"node_set: {node_set}")
    with open(file_path, 'w') as file:
        file.writelines('\n'.join(node_set))

In [10]:
type(yaml_data)


dict